<h1> Imports </h1>

In [19]:
import requests
import uuid
from datetime import datetime
from typing import Dict, Any, Optional, List
import random
import time

<h1> Estrutura Base para usar os endpoints de contexto </h1>

In [5]:
# Configuração base
BASE_URL = "http://212.85.1.27:8009/"  # Ajuste para sua URL

def extrair_telefones_periodo(
    data_inicio: Optional[str] = None,
    data_fim: Optional[str] = None
) -> Dict[str, Any]:
    """
    Extrai telefones que tiveram conversas no período
    
    Args:
        data_inicio: 'YYYY-MM-DD HH:MM:SS' ou None
        data_fim: 'YYYY-MM-DD HH:MM:SS' ou None
    """
    url = f"{BASE_URL}/context/extrair-telefones-conversas-periodo"
    payload = {}
    
    if data_inicio:
        payload["data_hora_inicio"] = data_inicio
    if data_fim:
        payload["data_hora_fim"] = data_fim
    
    response = requests.post(url, json=payload)
    return response.json()

def extrair_telefones_com_ia(
    data_inicio: Optional[str] = None,
    data_fim: Optional[str] = None
) -> Dict[str, Any]:
    """
    Extrai telefones que tiveram mensagens de IA no período
    """
    url = f"{BASE_URL}/context/extrair-telefones-conversas-periodo-com-ia"
    payload = {}
    
    if data_inicio:
        payload["data_hora_inicio"] = data_inicio
    if data_fim:
        payload["data_hora_fim"] = data_fim
    
    response = requests.post(url, json=payload)
    return response.json()

def buscar_conversa_numero(
    numero_telefone: str,
    data_inicio: Optional[str] = None,
    data_fim: Optional[str] = None,
    quebrar_conversas: bool = False,
    formato_saida: str = "texto"
) -> Dict[str, Any]:
    """
    Busca conversa de um número específico com fallback
    
    Args:
        numero_telefone: Número do WhatsApp
        formato_saida: "texto" ou "json"
    """
    url = f"{BASE_URL}/context/buscar-conversas-numero-com-fallback"
    payload = {
        "numero_telefone": numero_telefone,
        "quebrar_conversas": quebrar_conversas,
        "formato_saida": formato_saida,
        "forcar_evolution_apenas": True
    }
    
    if data_inicio:
        payload["data_hora_inicio"] = data_inicio
    if data_fim:
        payload["data_hora_fim"] = data_fim
    
    response = requests.post(url, json=payload)
    return response.json()

def analisar_quantitativo(
    numero_telefone: str,
    data_inicio: Optional[str] = None,
    data_fim: Optional[str] = None,
    teto_minutos: int = 30
) -> Dict[str, Any]:
    """
    Análise quantitativa de uma conversa
    """
    url = f"{BASE_URL}/context/analisar-conversa-atributos-quantitativos"
    payload = {
        "numero_telefone": numero_telefone,
        "teto_minutos": teto_minutos
    }
    
    if data_inicio:
        payload["data_hora_inicio"] = data_inicio
    if data_fim:
        payload["data_hora_fim"] = data_fim
    
    response = requests.post(url, json=payload)
    return response.json()

def analisar_qualitativo(
    numero_telefone: str,
    data_inicio: Optional[str] = None,
    data_fim: Optional[str] = None,
    campos_analise: Optional[List[str]] = None,
    temperatura: float = 0.1,
    modelo: Optional[str] = None
) -> Dict[str, Any]:
    """
    Análise qualitativa de uma conversa com LLM
    
    Args:
        campos_analise: Lista de campos para analisar ou None para padrão
    """
    url = f"{BASE_URL}/context/analisar-conversa-atributos-qualitativos"
    payload = {
        "numero_telefone": numero_telefone,
        "temperatura": temperatura
    }
    
    if data_inicio:
        payload["data_hora_inicio"] = data_inicio
    if data_fim:
        payload["data_hora_fim"] = data_fim
    if campos_analise:
        payload["campos_analise"] = campos_analise
    if modelo:
        payload["modelo"] = modelo
    
    response = requests.post(url, json=payload)
    return response.json()

def obter_memoria_contexto(
    numero_telefone: str,
    data_inicio: Optional[str] = None,
    data_fim: Optional[str] = None,
    teto_minutos: int = 30,
    temperatura: float = 0.1,
    modelo: Optional[str] = None
) -> Dict[str, Any]:
    """
    Análise completa (quantitativa + qualitativa fusionada)
    """
    url = f"{BASE_URL}/context/obter-memoria-contexto"
    payload = {
        "numero_telefone": numero_telefone,
        "teto_minutos": teto_minutos,
        "temperatura": temperatura
    }
    
    if data_inicio:
        payload["data_hora_inicio"] = data_inicio
    if data_fim:
        payload["data_hora_fim"] = data_fim
    if modelo:
        payload["modelo"] = modelo
    
    response = requests.post(url, json=payload)
    return response.json()

In [6]:
def schedule_birthday_task(
    target_datetime: str,
    phone_number: str, 
    nome_pessoa: str,
    tipo_fluxo: str = "normal",
    base_url: str = "http://212.85.1.27:8009",
    custom_task_id: Optional[str] = None
) -> Dict[str, Any]:
    """
    Agenda uma task de aniversário para execução automática
    
    Args:
        target_datetime (str): Data/hora de execução no formato "2025-12-25T09:00:00"
        phone_number (str): Número de telefone no formato "5531999999999"
        nome_pessoa (str): Nome da pessoa que fará aniversário
        tipo_fluxo (str): Tipo do fluxo ("normal" ou "final_de_semana")
        base_url (str): URL base da API (opcional)
        custom_task_id (str): ID personalizado da task (opcional)
    
    Returns:
        Dict com resultado do agendamento
        
    Examples:
        >>> # Agendar aniversário para amanhã às 9h
        >>> result = schedule_birthday_task(
        ...     target_datetime="2025-08-19T09:00:00",
        ...     phone_number="5531995655690", 
        ...     nome_pessoa="João Silva",
        ...     tipo_fluxo="normal"
        ... )
        >>> print(result["status"])
        "success"
        
        >>> # Agendar para final de semana
        >>> result = schedule_birthday_task(
        ...     target_datetime="2025-12-21T10:00:00",
        ...     phone_number="5531888888888",
        ...     nome_pessoa="Maria Santos", 
        ...     tipo_fluxo="final_de_semana"
        ... )
    """
    
    # Gerar ID único se não fornecido
    if not custom_task_id:
        task_id = f"aniversario_{nome_pessoa.replace(' ', '_').lower()}_{uuid.uuid4().hex[:8]}"
    else:
        task_id = custom_task_id
    
    # Montar payload da requisição
    payload = {
        "target_datetime": target_datetime,
        "task_type": "http_request", 
        "task_id": task_id,
        "description": f"Aniversário de {nome_pessoa} ({tipo_fluxo})",
        "task_params": {
            "url": f"{base_url}/activation/trigger",
            "method": "POST",
            "payload": {
                "phone_number": phone_number,
                "flow": "aniversarios",
                "informacoes_contexto": {
                    "nome_pessoa": nome_pessoa,
                    "tipo_fluxo": tipo_fluxo
                }
            }
        }
    }
    
    # URL do endpoint de agendamento
    schedule_url = f"{base_url}/tasks/schedule"
    
    print(f"🎂 [SCHEDULE-BIRTHDAY] Agendando aniversário de {nome_pessoa}")
    print(f"🎂 [SCHEDULE-BIRTHDAY] Para: {target_datetime}")
    print(f"🎂 [SCHEDULE-BIRTHDAY] Phone: {phone_number}")
    print(f"🎂 [SCHEDULE-BIRTHDAY] Tipo: {tipo_fluxo}")
    print(f"🎂 [SCHEDULE-BIRTHDAY] Task ID: {task_id}")
    
    try:
        # Fazer requisição
        response = requests.post(
            schedule_url,
            json=payload,
            headers={'Content-Type': 'application/json'},
            timeout=30
        )
        
        # Verificar se foi bem-sucedida
        if response.status_code == 200:
            result = response.json()
            
            print(f"✅ [SCHEDULE-BIRTHDAY] Sucesso! Task {task_id} agendada")
            print(f"✅ [SCHEDULE-BIRTHDAY] Próxima execução: {result.get('next_run', 'N/A')}")
            
            return {
                "status": "success",
                "task_id": task_id,
                "nome_pessoa": nome_pessoa,
                "phone_number": phone_number,
                "scheduled_for": target_datetime,
                "tipo_fluxo": tipo_fluxo,
                "api_response": result,
                "message": f"Aniversário de {nome_pessoa} agendado com sucesso!"
            }
        else:
            error_data = response.json() if response.content else {}
            
            print(f"❌ [SCHEDULE-BIRTHDAY] Erro HTTP {response.status_code}")
            print(f"❌ [SCHEDULE-BIRTHDAY] Resposta: {error_data}")
            
            return {
                "status": "error",
                "error": f"HTTP {response.status_code}",
                "task_id": task_id,
                "nome_pessoa": nome_pessoa,
                "phone_number": phone_number,
                "api_response": error_data,
                "message": f"Erro ao agendar aniversário de {nome_pessoa}"
            }
            
    except requests.exceptions.RequestException as e:
        print(f"❌ [SCHEDULE-BIRTHDAY] Erro de conexão: {e}")
        
        return {
            "status": "error",
            "error": str(e),
            "task_id": task_id,
            "nome_pessoa": nome_pessoa,
            "phone_number": phone_number,
            "message": f"Erro de conexão ao agendar aniversário de {nome_pessoa}"
        }
    
    except Exception as e:
        print(f"❌ [SCHEDULE-BIRTHDAY] Erro inesperado: {e}")
        
        return {
            "status": "error",
            "error": str(e),
            "task_id": task_id,
            "nome_pessoa": nome_pessoa,
            "phone_number": phone_number,
            "message": f"Erro inesperado ao agendar aniversário de {nome_pessoa}"
        }

<h1> Grupos </h1>

<h1> Análise </h1>

In [20]:
import requests

import requests

from urllib.parse import quote

import re

def fetch_all_groups(base_url: str, instance: str, api_key: str, get_participants: bool = False):
    """
    Busca todos os grupos de uma instância.
    """
    url = f"{base_url}/group/fetchAllGroups/{instance}"
    params = {"getParticipants": str(get_participants).lower()}
    headers = {
        "apikey": api_key,            # 👈 exatamente como no Postman
        "Content-Type": "application/json"
    }

    response = requests.get(url, headers=headers, params=params)

    # Se a API retornar 401, mostra detalhes
    if response.status_code == 401:
        raise Exception(f"Unauthorized (401). Verifique se o API Key está correto. "
                        f"URL: {response.url}, Headers enviados: {headers}")

    response.raise_for_status()
    return response.json()

def find_group_participants(base_url: str, instance: str, api_key: str, group_jid: str):
    """
    Busca os participantes de um grupo específico.

    :param base_url: URL base da API (ex: https://api.exemplo.com)
    :param instance: Identificador da instância
    :param api_key: Chave da API
    :param group_jid: JID do grupo (ex: "554899999999-123456@g.us")
    :return: Resposta JSON da API
    """
    # Codificar instance para garantir compatibilidade (ex: espaços)
    instance_encoded = quote(instance, safe='')
    
    url = f"{base_url}/group/participants/{instance_encoded}"
    params = {"groupJid": group_jid}
    headers = {
        "apikey": api_key,
        "Content-Type": "application/json"
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 401:
        raise Exception(f"Unauthorized (401). Verifique se o API Key está correto. "
                        f"URL: {response.url}, Headers enviados: {headers}")

    response.raise_for_status()
    return response.json()

def find_messages(base_url, instance, api_key, remote_jid):
   """
   Busca mensagens na Evolution API
   
   Args:
       base_url (str): URL base da API (ex: 'https://api.exemplo.com')
       instance (str): Nome da instância
       api_key (str): Chave da API
       remote_jid (str): ID remoto do contato/grupo
   
   Returns:
       dict: Resposta da API
   """
   
   url = f"{base_url}/chat/findMessages/{instance}"
   
   headers = {
       "apikey": api_key,
       "Content-Type": "application/json"
   }
   
   body = {
       "where": {
           "key": {
               "remoteJid": remote_jid
           }
       }
   }
   
   try:
       response = requests.post(url, headers=headers, json=body)
       response.raise_for_status()
       return response.json()
   
   except requests.exceptions.RequestException as e:
       print(f"Erro na requisição: {e}")
       return None

# Exemplo de uso:
# result = find_messages("https://sua-api.com", "minha_instancia", "sua_api_key", "5511999999999@s.whatsapp.net")

def format_brazilian_phone_to_whatsapp(phone_number):
    clean_number = ''.join(filter(str.isdigit, phone_number))
    
    # Se tem 13 dígitos, remove o 5º dígito (9 extra)
    if len(clean_number) == 13 and clean_number.startswith('55'):
        formatted_number = clean_number[:4] + clean_number[5:]
    else:
        formatted_number = clean_number
    
    return f"{formatted_number}@s.whatsapp.net"

def limpar_numero_whatsapp(numero: str) -> str:
    """Converte '553195655690@s.whatsapp.net' para '5531995655690'."""
    numero_limpo = numero.split("@")[0]  # remove @s.whatsapp.net
    if len(numero_limpo) > 4 and numero_limpo[4] != "9":
        numero_limpo = numero_limpo[:4] + "9" + numero_limpo[4:]
    return numero_limpo

def find_correct_brazilian_phone_format(base_url, instance, api_key, phone_number):
    """
    Descobre o formato correto do número brasileiro para a Evolution API
    
    Returns:
        str: remoteJid no formato correto ou None se não encontrar mensagens
    """
    
    def find_messages(remote_jid):
        import requests
        from urllib.parse import quote
        
        encoded_instance = quote(instance)
        url = f"{base_url}/chat/findMessages/{encoded_instance}"
        
        headers = {
            "apikey": api_key,
            "Content-Type": "application/json"
        }
        
        body = {
            "where": {
                "key": {
                    "remoteJid": remote_jid
                }
            }
        }
        
        try:
            response = requests.post(url, headers=headers, json=body)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Erro na requisição para {remote_jid}: {e}")
            return None
    
    clean_number = ''.join(filter(str.isdigit, phone_number))
    
    # Define os formatos a testar
    formats = []
    if len(clean_number) == 13 and clean_number.startswith('55'):
        formats = [
            f"{clean_number}@s.whatsapp.net",  # 13 dígitos primeiro
            f"{clean_number[:4]}{clean_number[5:]}@s.whatsapp.net"  # 12 dígitos
        ]
    else:
        formats = [f"{clean_number}@s.whatsapp.net"]
    
    # Testa cada formato
    for remote_jid in formats:
        result = find_messages(remote_jid)
        if result and result.get('messages', {}).get('records') and len(result['messages']['records']) > 0:
            return remote_jid  # Retorna apenas o formato correto
    
    return None  # Não encontrou mensagens em nenhum formato

def check_phone_has_messages_raw(base_url, instance, api_key, phone_number):
    """
    Verifica se um número tem mensagens SEM fazer conversão de formato
    Testa o número exatamente como foi fornecido
    
    Args:
        base_url (str): URL base da API
        instance (str): Nome da instância
        api_key (str): Chave da API
        phone_number (str): Número de telefone (sem conversão)
    
    Returns:
        bool: True se encontrar mensagens, False caso contrário
    """
    
    def find_messages(remote_jid):
        """
        Inner function para buscar mensagens na Evolution API
        """
        import requests
        from urllib.parse import quote
        
        encoded_instance = quote(instance)
        url = f"{base_url}/chat/findMessages/{encoded_instance}"
        
        headers = {
            "apikey": api_key,
            "Content-Type": "application/json"
        }
        
        body = {
            "where": {
                "key": {
                    "remoteJid": remote_jid
                }
            }
        }
        
        try:
            response = requests.post(url, headers=headers, json=body)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException:
            return None
    
    # Usa o número exatamente como foi fornecido + @s.whatsapp.net
    clean_number = ''.join(filter(str.isdigit, phone_number))
    remote_jid = f"{clean_number}@s.whatsapp.net"
    
    # Testa apenas o formato fornecido, sem conversões
    result = find_messages(remote_jid)
    if result and result.get('messages', {}).get('records') and len(result['messages']['records']) > 0:
        return True
    
    return False


import requests
import json

def send_message(numero: str, mensagem: str) -> dict:
    url = "http://212.85.1.27:8025/chat/send"
    headers = {"Content-Type": "application/json"}
    payload = {
        "phone_number": numero,
        "message": mensagem,
        "type": "text"
    }

    # aqui forçamos a serialização como no curl -d '...'
    response = requests.post(url, data=json.dumps(payload), headers=headers)
    
    try:
        return response.json()
    except Exception:
        return {"status": response.status_code, "response": response.text}

# Agora testando seu exemplo:
# check_phone_has_messages_raw(..., "5531995655690")  # → Deveria ser False
# check_phone_has_messages_raw(..., "553195655690")   # → Deveria ser True

In [21]:
resultados = fetch_all_groups("https://evolutionapi.gbstudios.tech",
                 "Fa Maia Comercial",
                 "VpN4MchD6Y0Z7sWRNHSeViu8emRQR3zm")

In [22]:
resultados

[{'id': '120363416579819696@g.us',
  'subject': 'Comunidade Tô Rica',
  'subjectOwner': '553798694711@s.whatsapp.net',
  'subjectTime': 1743685694,
  'pictureUrl': None,
  'size': 4,
  'creation': 1743685694,
  'owner': '553798694711@s.whatsapp.net',
  'desc': 'Comunidade Tô Rica',
  'descId': '10100.64064-332',
  'restrict': False,
  'announce': False,
  'isCommunity': True,
  'isCommunityAnnounce': False},
 {'id': '120363338050597260@g.us',
  'subject': 'Mindfullnes ✨️ Num Instante',
  'subjectOwner': '553198861210@s.whatsapp.net',
  'subjectTime': 1730720420,
  'pictureUrl': None,
  'size': 1,
  'creation': 1730420716,
  'owner': '553198861210@s.whatsapp.net',
  'desc': 'Mindfulness dia a dia, com Dra. Thaís Santos.',
  'descId': '846418167FEA4BCAF00339F4FA847661',
  'restrict': False,
  'announce': False,
  'isCommunity': True,
  'isCommunityAnnounce': False},
 {'id': '120363267261091493@g.us',
  'subject': 'RBC NETWORKING',
  'subjectOwner': '553192670015@s.whatsapp.net',
  'subje

In [23]:

for resultado in resultados:
    print(resultado['subject'], resultado['id'])


Comunidade Tô Rica 120363416579819696@g.us
Mindfullnes ✨️ Num Instante 120363338050597260@g.us
RBC NETWORKING 120363267261091493@g.us
Clube de Descontos PURE Blends (Comunidade Exclusiva Para Clientes Pure Blends) 2 120363183033245439@g.us
GCRIE 120363138329335509@g.us
COMUNIDADE N 120363029089686299@g.us
Team Comunidade 553199586356-1625754390@g.us
PROVA SOCIAL SAFIRA 553199586356-1620831995@g.us
LINK WEBNARIO HOJE 15HRS 553199586356-1606138252@g.us
LANÇAMENTO MARCA FM 120363040979925061@g.us
FOTOS LUAN 553199586356-1622290293@g.us
😍PASTA ESPECIAL😍 TRABALHO 553199586356-1604746828@g.us
VENDAS 120363040020551019@g.us
VIDEO ENCONTRO 120363020382343961@g.us
SAFIRA ACELERAÇÃO 2023 120363044762317191@g.us
ALAVANCADA SAFIRA - FALTAM 56 DIAS 120363044266045451@g.us
#EUSOUSAFIRA 💎 5521973093879-1620396297@g.us
PLANEJA 553194929825-1625607459@g.us
CONSULTORIA ROYAL PRESTIGE  120363301807227576@g.us
Palestrantes da Noite 120363326293571862@g.us
[ GT MARKETING ] - Fábricia 120363319589600946@g.u

In [24]:
participantes_grupo = find_group_participants("https://evolutionapi.gbstudios.tech",
                 "Fa Maia Comercial",
                 "VpN4MchD6Y0Z7sWRNHSeViu8emRQR3zm",
                 "120363344594416817@g.us")

In [25]:
participantes_grupo

{'participants': [{'id': '553199101795@s.whatsapp.net',
   'jid': '553199101795@s.whatsapp.net',
   'lid': '42537490354178@lid',
   'admin': None,
   'name': 'Luciana Prates',
   'imgUrl': 'https://pps.whatsapp.net/v/t61.24694-24/505186310_1670592770282141_2133193815112404566_n.jpg?ccb=11-4&oh=01_Q5Aa2QGY6miZiobfJ5-Uo1OSQEM9KyduagVb5xayQJjErORlfQ&oe=68BC5206&_nc_sid=5e03e0&_nc_cat=103'},
  {'id': '553186774435@s.whatsapp.net',
   'jid': '553186774435@s.whatsapp.net',
   'lid': '263011012985082@lid',
   'admin': None,
   'name': 'Roselayne',
   'imgUrl': 'https://pps.whatsapp.net/v/t61.24694-24/491841935_1372334223794067_4794234634606846587_n.jpg?ccb=11-4&oh=01_Q5Aa2QGMWti2Xhrn1mhlwUnMjtX1UYZFn2wts6GExwi49LY8DA&oe=68BCAB76&_nc_sid=5e03e0&_nc_cat=100'},
  {'id': '553185051528@s.whatsapp.net',
   'jid': '553185051528@s.whatsapp.net',
   'lid': '195202035806413@lid',
   'admin': None},
  {'id': '553188858481@s.whatsapp.net',
   'jid': '553188858481@s.whatsapp.net',
   'lid': '2135115316757

<h1> Números que a Letícia Enviou/Ativou </h1>

In [26]:
telefones_ja_ativados = [
    "5527992027535",
    "5524981604432",
    "553196856817",
    "553196161702",
    "553197733587",
    "553190861806",
    "555599666832",
    "553172144220",
    "553183993170",
    "553188683543",
    "5527997374177",
    "553171487572",
    "553171700813",
    "553173438496",
    "553173451644",
    "553183160550",
    "553185588939",
    "553188929472",
    "553189027712",
    "553190670771",
    "553191004550",
    "553192125169",
    "553194667082",
    "553195693255",
    "553197013863",
    "553197960044",
    "553198795412",
    "556181353474",
    "553199823391",
    "553199924241",
    "553288071604",
    "553598125948",
    "555195000146",
    "555195056289",
]


In [27]:
len(telefones_ja_ativados)

34

In [28]:
find_correct_brazilian_phone_format("https://evolutionapi.gbstudios.tech",
                 "Fa Maia Comercial",
                 "VpN4MchD6Y0Z7sWRNHSeViu8emRQR3zm", "5531995655690")

'553195655690@s.whatsapp.net'

<h1> Total Base de Contatos do Grupo MasterClass </h1>

In [29]:
telefones_convertidos_jid = [find_correct_brazilian_phone_format("https://evolutionapi.gbstudios.tech",
                 "Fa Maia Comercial",
                 "VpN4MchD6Y0Z7sWRNHSeViu8emRQR3zm", tel) for tel in telefones_ja_ativados]

dados_participantes = {dados.get('id'): {'name': dados.get('name', None),
                                         }
                       for dados in participantes_grupo['participants']}

dados_participantes_filtrado = {
    jid: info
    for jid, info in dados_participantes.items()
    if jid not in telefones_convertidos_jid
}


In [30]:
dados_participantes_filtrado

{'553199101795@s.whatsapp.net': {'name': 'Luciana Prates'},
 '553186774435@s.whatsapp.net': {'name': 'Roselayne'},
 '553185051528@s.whatsapp.net': {'name': None},
 '553188858481@s.whatsapp.net': {'name': None},
 '553194932057@s.whatsapp.net': {'name': None},
 '553171049467@s.whatsapp.net': {'name': None},
 '553299799717@s.whatsapp.net': {'name': None},
 '553888093291@s.whatsapp.net': {'name': 'Rosy Comunidade '},
 '5524999938238@s.whatsapp.net': {'name': None},
 '553192058756@s.whatsapp.net': {'name': 'Dani To Rica'},
 '553197766144@s.whatsapp.net': {'name': None},
 '553194982527@s.whatsapp.net': {'name': 'Ana Beatriz'},
 '553191913561@s.whatsapp.net': {'name': 'Neide Comam'},
 '553180213274@s.whatsapp.net': {'name': 'Andrezza Martins'},
 '553183249012@s.whatsapp.net': {'name': 'Flavia Nunes'},
 '553194321566@s.whatsapp.net': {'name': 'Valeria Alves de Melo'},
 '553799114930@s.whatsapp.net': {'name': 'Val'},
 '553186975727@s.whatsapp.net': {'name': None},
 '553191484386@s.whatsapp.net'

<h1> Ativação MasterClass </h1>

In [53]:
def processar_contatos_whatsapp(
    dados_whatsapp, 
    filtro="todos",
    mostrar_estatisticas=True
):
    """
    Processa contatos do WhatsApp e converte para formato de lote de ativação
    
    Args:
        dados_whatsapp: Dict no formato {'numero@s.whatsapp.net': {'name': 'Nome'}}
        filtro: "todos" | "apenas_com_nome" | "apenas_sem_nome"
        mostrar_estatisticas: Se True, imprime estatísticas
    
    Returns:
        Lista de contatos no formato para disparar_lote_distribuido()
    """
    
    # Converter todos os contatos
    contatos_convertidos = []
    
    for whatsapp_id, dados in dados_whatsapp.items():
        phone_number = whatsapp_id
        nome = dados.get('name')
        
        # Limpar o nome
        if nome is None or nome.strip() == '':
            # Sem nome - apenas phone_number
            contato = {"phone_number": phone_number}
        else:
            # Com nome - incluir ambos os campos
            contato = {
                "phone_number": phone_number,
                "nome": nome.strip()
            }
        
        contatos_convertidos.append(contato)
    
    # Calcular estatísticas
    total = len(contatos_convertidos)
    com_nome = len([c for c in contatos_convertidos if 'nome' in c and c.get('nome', '').strip() != ''])
    sem_nome = total - com_nome
    
    # Aplicar filtro
    if filtro == "apenas_com_nome":
        contatos_filtrados = [c for c in contatos_convertidos if 'nome' in c and c.get('nome', '').strip() != '']
    elif filtro == "apenas_sem_nome":
        contatos_filtrados = [c for c in contatos_convertidos if not ('nome' in c and c.get('nome', '').strip() != '')]
    else:  # "todos"
        contatos_filtrados = contatos_convertidos
    
    # Mostrar estatísticas se solicitado
    if mostrar_estatisticas:
        percentual_com_nome = round((com_nome / total * 100), 1) if total > 0 else 0
        
        print(f"\n=== ESTATÍSTICAS DOS CONTATOS ===")
        print(f"Total de contatos: {total}")
        print(f"Com nome: {com_nome} ({percentual_com_nome}%)")
        print(f"Sem nome: {sem_nome} ({100-percentual_com_nome}%)")
        print(f"Filtro aplicado: {filtro}")
        print(f"Contatos após filtro: {len(contatos_filtrados)}")
        print(f"================================\n")
    
    return contatos_filtrados

def disparar_ativacao_masterclass(
    phone_number: str, 
    nome: Optional[str] = None,
    base_url: str = "http://212.85.1.27:8025"
) -> Dict[str, Any]:
    """
    Dispara ativação do Masterclass da Fabrícia
    
    Args:
        phone_number: Telefone no formato brasileiro (5531999999999)
        nome: Nome da pessoa (opcional, se None ou vazio usa mensagem genérica)
        base_url: URL base da API (padrão: http://212.85.1.27:8025)
    
    Returns:
        Dict com resposta da API ou informações de erro
    """
    
    # Validar telefone básico
    if not phone_number.startswith('55'):
        return {
            "error": True,
            "message": "Número deve começar com 55 (código do Brasil)"
        }
    
    if len(phone_number) not in [12, 13]:
        return {
            "error": True,
            "message": "Formato de telefone inválido (deve ter 12 ou 13 dígitos)"
        }
    
    # Preparar payload
    payload = {
        "phone_number": phone_number,
        "flow": "ativacao_masterclass",
        "informacoes_contexto": {
            "nome": nome if nome else ""  # Garantir que seja string vazia se None
        }
    }
    
    # URL completa
    url = f"{base_url}/activation/trigger"
    
    # Headers
    headers = {
        "Content-Type": "application/json"
    }
    
    try:
        print(f"[TRIGGER_FUNC] Enviando para: {phone_number}")
        print(f"[TRIGGER_FUNC] Nome: '{nome}'" if nome else "[TRIGGER_FUNC] Sem nome")
        
        # Fazer requisição
        response = requests.post(
            url=url,
            headers=headers,
            json=payload,
            timeout=30  # 30 segundos timeout
        )
        
        # Parse da resposta
        if response.status_code == 200:
            result = response.json()
            print(f"[TRIGGER_FUNC] Sucesso: {result.get('message', 'OK')}")
            return {
                "error": False,
                "status_code": response.status_code,
                "data": result
            }
        else:
            print(f"[TRIGGER_FUNC] Erro HTTP {response.status_code}")
            try:
                error_data = response.json()
            except:
                error_data = {"message": response.text}
                
            return {
                "error": True,
                "status_code": response.status_code,
                "message": error_data.get("error", "Erro desconhecido"),
                "data": error_data
            }
            
    except requests.exceptions.Timeout:
        return {
            "error": True,
            "message": "Timeout na requisição (30s)"
        }
    except requests.exceptions.ConnectionError:
        return {
            "error": True,
            "message": "Erro de conexão com a API"
        }
    except Exception as e:
        return {
            "error": True,
            "message": f"Erro inesperado: {str(e)}"
        }

import requests
import json
import time
import random
from typing import List, Dict, Any, Optional
from datetime import datetime

def disparar_lote_distribuido(
    contatos: List[Dict[str, str]],
    tempo_total_segundos: int,
    base_url: str = "http://212.85.1.27:8025"
) -> Dict[str, Any]:
    """
    Dispara ativações em lote com distribuição temporal irregular
    
    Args:
        contatos: Lista de dicts com 'phone_number' e opcionalmente 'nome'
                  Exemplo: [{"phone_number": "5531999999999", "nome": "João"}, 
                           {"phone_number": "5531888888888"}]
        tempo_total_segundos: Tempo total em segundos para distribuir os envios
        base_url: URL base da API
    
    Returns:
        Dict com resumo dos envios e timeline executada
    """
    
    def gerar_intervalos_irregulares(total_segundos: int, quantidade: int) -> List[float]:
        """Gera intervalos irregulares que somam o tempo total"""
        if quantidade <= 1:
            return [0]
        
        # Gerar pontos aleatórios no tempo total
        pontos = sorted([random.uniform(0, total_segundos) for _ in range(quantidade - 1)])
        
        # Calcular intervalos entre os pontos
        intervalos = []
        ultimo_ponto = 0
        
        for ponto in pontos:
            intervalo = ponto - ultimo_ponto
            intervalos.append(intervalo)
            ultimo_ponto = ponto
        
        # Adicionar intervalo final
        intervalos.append(total_segundos - ultimo_ponto)
        
        # O primeiro envio é imediato
        intervalos[0] = 0
        
        return intervalos
    
    def enviar_individual(phone_number: str, nome: Optional[str] = None) -> Dict[str, Any]:
        """Envia ativação individual"""
        
        # Processar número para formato que a API aceita
        if phone_number.endswith('@s.whatsapp.net'):
            numero_limpo = phone_number.replace('@s.whatsapp.net', '')
        else:
            numero_limpo = phone_number
        
        # Validações básicas
        if not numero_limpo.startswith('55'):
            return {"error": True, "message": "Número deve começar com 55 (código do Brasil)"}
        
        if len(numero_limpo) not in [12, 13]:
            return {"error": True, "message": "Formato de telefone inválido"}
        
        # Preparar payload com número limpo (sem @s.whatsapp.net)
        payload = {
            "phone_number": numero_limpo,  # API espera formato sem @s.whatsapp.net
            "flow": "ativacao_masterclass",
            "informacoes_contexto": {"nome": nome if nome else ""}
        }
        
        url = f"{base_url}/activation/trigger"
        headers = {"Content-Type": "application/json"}
        
        try:
            response = requests.post(url=url, headers=headers, json=payload, timeout=30)
            
            if response.status_code == 200:
                result = response.json()
                return {"error": False, "status_code": response.status_code, "data": result}
            else:
                try:
                    error_data = response.json()
                    error_message = error_data.get("error", "Erro da API")
                except:
                    error_message = f"HTTP {response.status_code}: {response.text}"
                
                return {
                    "error": True, 
                    "status_code": response.status_code, 
                    "message": error_message,
                    "data": error_data if 'error_data' in locals() else {}
                }
                
        except Exception as e:
            return {"error": True, "message": f"Erro de requisição: {str(e)}"}
    
    # Validação inicial
    if not contatos:
        return {"error": True, "message": "Lista de contatos vazia"}
    
    if tempo_total_segundos < 0:
        return {"error": True, "message": "Tempo total deve ser positivo"}
    
    # Gerar intervalos irregulares
    intervalos = gerar_intervalos_irregulares(tempo_total_segundos, len(contatos))
    
    # Inicializar resultado
    resultado = {
        "total": len(contatos),
        "sucessos": 0,
        "erros": 0,
        "tempo_total_planejado": tempo_total_segundos,
        "tempo_total_executado": 0,
        "inicio": datetime.now().isoformat(),
        "fim": None,
        "intervalos_gerados": intervalos,
        "timeline": [],
        "detalhes": []
    }
    
    print(f"[LOTE_DISTRIBUIDO] Iniciando envio de {len(contatos)} contatos")
    print(f"[LOTE_DISTRIBUIDO] Tempo total planejado: {tempo_total_segundos}s ({tempo_total_segundos/60:.1f} min)")
    print(f"[LOTE_DISTRIBUIDO] Intervalos gerados: {[f'{i:.1f}s' for i in intervalos[:5]]}{'...' if len(intervalos) > 5 else ''}")
    
    tempo_inicio = time.time()
    
    for i, contato in enumerate(contatos):
        # Aguardar o intervalo (exceto no primeiro)
        if i > 0:
            intervalo = intervalos[i]
            print(f"\n[LOTE_DISTRIBUIDO] Aguardando {intervalo:.1f}s antes do próximo envio...")
            time.sleep(intervalo)
        
        # Extrair dados do contato
        phone = contato.get("phone_number", "")
        nome = contato.get("nome", "")
        
        tempo_atual = time.time()
        tempo_decorrido = tempo_atual - tempo_inicio
        
        print(f"\n[LOTE_DISTRIBUIDO] {i+1}/{len(contatos)} - Enviando para {phone}")
        print(f"[LOTE_DISTRIBUIDO] Nome: '{nome}'" if nome else "[LOTE_DISTRIBUIDO] Sem nome")
        print(f"[LOTE_DISTRIBUIDO] Tempo decorrido: {tempo_decorrido:.1f}s")
        
        # Registrar timeline
        timeline_entry = {
            "indice": i + 1,
            "phone_number": phone,
            "nome": nome,
            "tempo_decorrido": round(tempo_decorrido, 1),
            "intervalo_anterior": round(intervalos[i], 1) if i > 0 else 0,
            "timestamp": datetime.now().isoformat()
        }
        
        if not phone:
            # Erro de validação
            detalhes = {
                "phone_number": phone,
                "nome": nome,
                "erro": True,
                "message": "Telefone não informado",
                "tempo_decorrido": round(tempo_decorrido, 1)
            }
            resultado["erros"] += 1
            timeline_entry["status"] = "erro"
            timeline_entry["message"] = "Telefone não informado"
        else:
            # Tentar enviar
            response = enviar_individual(phone_number=phone, nome=nome)
            
            detalhes = {
                "phone_number": phone,
                "nome": nome,
                "erro": response.get("error", False),
                "message": response.get("message", "OK"),
                "status_code": response.get("status_code"),
                "tempo_decorrido": round(tempo_decorrido, 1)
            }
            
            if response.get("error", False):
                resultado["erros"] += 1
                timeline_entry["status"] = "erro"
                timeline_entry["message"] = response.get("message", "Erro desconhecido")
            else:
                resultado["sucessos"] += 1
                timeline_entry["status"] = "sucesso"
                timeline_entry["message"] = "Enviado com sucesso"
        
        resultado["detalhes"].append(detalhes)
        resultado["timeline"].append(timeline_entry)
    
    # Finalizar
    tempo_total_executado = time.time() - tempo_inicio
    resultado["tempo_total_executado"] = round(tempo_total_executado, 1)
    resultado["fim"] = datetime.now().isoformat()
    
    print(f"\n[LOTE_DISTRIBUIDO] CONCLUÍDO!")
    print(f"[LOTE_DISTRIBUIDO] Sucessos: {resultado['sucessos']}")
    print(f"[LOTE_DISTRIBUIDO] Erros: {resultado['erros']}")
    print(f"[LOTE_DISTRIBUIDO] Tempo planejado: {tempo_total_segundos}s")
    print(f"[LOTE_DISTRIBUIDO] Tempo executado: {tempo_total_executado:.1f}s")
    print(f"[LOTE_DISTRIBUIDO] Diferença: {abs(tempo_total_executado - tempo_total_segundos):.1f}s")
    
    return resultado


In [16]:
contatos = [
    {"phone_number": "5531995655690"},
    {"phone_number": "5531991258669", 'nome': 'MD'},  # Sem nome
]

In [35]:
contatos_input = processar_contatos_whatsapp(dados_participantes_filtrado)


=== ESTATÍSTICAS DOS CONTATOS ===
Total de contatos: 170
Com nome: 75 (44.1%)
Sem nome: 95 (55.9%)
Filtro aplicado: todos
Contatos após filtro: 170



In [38]:
len(dados_participantes_filtrado)

170

In [37]:
len(contatos_input)

170

In [44]:
contatos_input[0].get('phone_number')

'553199101795@s.whatsapp.net'

In [45]:
disparar_ativacao_masterclass(contatos_input[0].get('phone_number'), contatos_input[0].get('nome'))

{'error': True,
 'message': 'Formato de telefone inválido (deve ter 12 ou 13 dígitos)'}

In [43]:
contatos_input

[{'phone_number': '553199101795@s.whatsapp.net', 'nome': 'Luciana Prates'},
 {'phone_number': '553186774435@s.whatsapp.net', 'nome': 'Roselayne'},
 {'phone_number': '553185051528@s.whatsapp.net'},
 {'phone_number': '553188858481@s.whatsapp.net'},
 {'phone_number': '553194932057@s.whatsapp.net'},
 {'phone_number': '553171049467@s.whatsapp.net'},
 {'phone_number': '553299799717@s.whatsapp.net'},
 {'phone_number': '553888093291@s.whatsapp.net', 'nome': 'Rosy Comunidade'},
 {'phone_number': '5524999938238@s.whatsapp.net'},
 {'phone_number': '553192058756@s.whatsapp.net', 'nome': 'Dani To Rica'},
 {'phone_number': '553197766144@s.whatsapp.net'},
 {'phone_number': '553194982527@s.whatsapp.net', 'nome': 'Ana Beatriz'},
 {'phone_number': '553191913561@s.whatsapp.net', 'nome': 'Neide Comam'},
 {'phone_number': '553180213274@s.whatsapp.net', 'nome': 'Andrezza Martins'},
 {'phone_number': '553183249012@s.whatsapp.net', 'nome': 'Flavia Nunes'},
 {'phone_number': '553194321566@s.whatsapp.net',
  'n

In [57]:
disparar_lote_distribuido(contatos_input[2:], 5400)

[LOTE_DISTRIBUIDO] Iniciando envio de 168 contatos
[LOTE_DISTRIBUIDO] Tempo total planejado: 5400s (90.0 min)
[LOTE_DISTRIBUIDO] Intervalos gerados: ['0.0s', '24.7s', '12.2s', '98.3s', '82.0s']...

[LOTE_DISTRIBUIDO] 1/168 - Enviando para 553185051528@s.whatsapp.net
[LOTE_DISTRIBUIDO] Sem nome
[LOTE_DISTRIBUIDO] Tempo decorrido: 0.0s

[LOTE_DISTRIBUIDO] Aguardando 24.7s antes do próximo envio...

[LOTE_DISTRIBUIDO] 2/168 - Enviando para 553188858481@s.whatsapp.net
[LOTE_DISTRIBUIDO] Sem nome
[LOTE_DISTRIBUIDO] Tempo decorrido: 26.0s

[LOTE_DISTRIBUIDO] Aguardando 12.2s antes do próximo envio...

[LOTE_DISTRIBUIDO] 3/168 - Enviando para 553194932057@s.whatsapp.net
[LOTE_DISTRIBUIDO] Sem nome
[LOTE_DISTRIBUIDO] Tempo decorrido: 38.9s

[LOTE_DISTRIBUIDO] Aguardando 98.3s antes do próximo envio...

[LOTE_DISTRIBUIDO] 4/168 - Enviando para 553171049467@s.whatsapp.net
[LOTE_DISTRIBUIDO] Sem nome
[LOTE_DISTRIBUIDO] Tempo decorrido: 138.0s

[LOTE_DISTRIBUIDO] Aguardando 82.0s antes do próximo 

{'total': 168,
 'sucessos': 168,
 'erros': 0,
 'tempo_total_planejado': 5400,
 'tempo_total_executado': 5537.8,
 'inicio': '2025-08-28T12:48:00.809464',
 'fim': '2025-08-28T14:20:18.646222',
 'intervalos_gerados': [0,
  24.738988803780444,
  12.207975629628919,
  98.32362438085767,
  82.02380804816539,
  19.03835856676227,
  6.794830969054544,
  34.8088592272685,
  53.643941960795075,
  23.930865088857274,
  19.677688200553348,
  46.19762904925636,
  36.49424643195135,
  36.62346777221984,
  8.427269561443268,
  13.505068563335158,
  27.75998935196094,
  81.3611626416789,
  38.227709840813986,
  7.949764594663861,
  60.033847930865136,
  5.332000465533156,
  9.63203229730027,
  9.969915675768107,
  1.4347250871827555,
  26.061772806487056,
  20.549271812368715,
  67.53010234270812,
  208.57101134241896,
  9.930290059535082,
  48.965385674849585,
  11.94273417482782,
  14.509528069719181,
  4.790729972755344,
  0.06418575741804489,
  2.8808516294611763,
  7.481364323780554,
  2.14577275